In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric, Features, ClassLabel, Value

trans_labels = ["POS","NEG","NEU"]

trans = {
    'POS': 0,
    'NEG': 1,
    'NEU': 2
}

features = Features({
    'index': Value(dtype='uint64'),
    'id_news': Value(dtype='uint64'),
    'title': Value(dtype='string'),
    'label': ClassLabel(names=trans_labels)
})

raw_data = load_dataset(
    'csv', data_files='./data/new_dataset.csv', split='train',
    skiprows=[0],
    column_names=["index", "id_news", "title", "label"],
    features=features
)

raw_data = raw_data.rename_column('title', 'text')
raw_data = raw_data.train_test_split(0.2, 0.8)


/home/usuariop/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-fdb4a1a13dbc4e72
Found cached dataset csv (/home/usuariop/.cache/huggingface/datasets/csv/default-fdb4a1a13dbc4e72/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [29]:
for i in range(29, 200):
    row = raw_data["train"][i]
    if row["label"] == 1:
        print(i, row)
        break

30 {'index': 15, 'id_news': 21866218, 'text': 'Graves lesiones de guardia de parque municipal de Osorno tras violenta agresión', 'label': 1}


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "finiteautomata/beto-sentiment-analysis"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_data.map(tokenize_function, batched=True)

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

100%|██████████| 1/1 [00:00<00:00, 70.29ba/s]


In [4]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['index', 'id_news', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 262
    })
    test: Dataset({
        features: ['index', 'id_news', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 66
    })
})

In [5]:
# Example:
nlp("Esto es una prueba larga de algo positivo")

[{'label': 'POS', 'score': 0.9894223809242249}]

In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(262))

small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(66))

In [7]:
from transformers import Trainer, TrainingArguments
import torch

def compute_metrics(eval_pred):
    print("Computing!")
    metrics = ["accuracy"]#, "f1"]#, "precision", "recall"]#, "f1"]# List of metrics to return
    metric = {}
    for met in metrics:
        metric[met] = load_metric(met)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric_res = {}
    for met in metrics:
        metric_res[met] = metric[met].compute(
            predictions=predictions, references=labels)[met]
    return metric_res


training_args = TrainingArguments(
    "test_trainer", label_names=trans_labels)

#model#.to(torch.device("cpu"))

training_args = TrainingArguments(
    "test_trainer",
    #label_names=['POS', 'NEG', 'NEU'],
    # output_dir="results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    # disable_tqdm=False,
    #logging_steps=
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
#trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, index, id_news. If text, index, id_news are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 8
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
 89%|████████▉ | 8/9 [00:02<00:00,  3.39it/s]/tmp/ipykernel_33772/19635434.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric[met] = load_metric(met)


Computing!


100%|██████████| 9/9 [00:03<00:00,  2.88it/s]


{'eval_loss': 4.423174858093262,
 'eval_accuracy': 0.15151515151515152,
 'eval_runtime': 3.9351,
 'eval_samples_per_second': 16.772,
 'eval_steps_per_second': 2.287}

In [8]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, index, id_news. If text, index, id_news are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/usuariop/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 262
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 660
  Number of trainable parameters = 109856259
 76%|███████▌  | 500/660 [03:48<01:13,  2.17it/s]Saving model checkpoint to test_trainer/checkpo

{'loss': 0.3309, 'learning_rate': 1.2121212121212122e-05, 'epoch': 7.58}


Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
100%|██████████| 660/660 [05:04<00:00,  2.47it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 660/660 [05:04<00:00,  2.17it/s]

{'train_runtime': 304.078, 'train_samples_per_second': 8.616, 'train_steps_per_second': 2.17, 'train_loss': 0.25082253083360917, 'epoch': 10.0}


TrainOutput(global_step=660, training_loss=0.25082253083360917, metrics={'train_runtime': 304.078, 'train_samples_per_second': 8.616, 'train_steps_per_second': 2.17, 'train_loss': 0.25082253083360917, 'epoch': 10.0})

In [14]:
trainer.save_model("./saved_model/")

Saving model checkpoint to ./saved_model/
Configuration saved in ./saved_model/config.json
Model weights saved in ./saved_model/pytorch_model.bin
tokenizer config file saved in ./saved_model/tokenizer_config.json
Special tokens file saved in ./saved_model/special_tokens_map.json


In [15]:
res = trainer.evaluate()
res

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, index, id_news. If text, index, id_news are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 8


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)

## Metricas

In [16]:
from metrics import get_confusion_matrix

In [17]:
### Predictions of the train-dataset ###

model.to(torch.device("cpu"))
human_labels = raw_data["train"]["label"]

#text = torch.tensor(raw_data["train"]["text"])
text = raw_data["train"]["text"]
pred_labels = nlp(text)
pred_labels = [trans[row["label"]] for row in pred_labels]

In [18]:
m = get_confusion_matrix(human_labels, pred_labels)
print("      " + "  ".join(n for n in trans_labels))
for label, l in zip(trans_labels, m):
    print(f"{label} [{l[0]:4d},{l[1]:4d},{l[2]:4d}]")


      POS  NEG  NEU
POS [   0,  66,   0]
NEG [   0,   0,  28]
NEU [ 168,   0,   0]
